## Text Analytics Group Assignment 2
### ArjunVarma_AnyingLi_YuanzhuoPeng_JiaqiuWang_NimishAmlathe

In [1]:
from pandas import Series, DataFrame
import pandas as pd
from patsy import dmatrices
%pylab inline

/Users/pranjor/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Populating the interactive namespace from numpy and matplotlib


In [3]:
yelp = pd.read_csv('Yelp Data Restaurant Reviews Ratings.csv')
yelp['rating'] = yelp['stars'].map(lambda x: 'High' if x >3  else 'Low')
yelp[:5]

,stars,votes_cool,votes_funny,votes_useful,Cheap,Moderate,Expensive,VeryExpensive,American,Chinese,...,Indian,Italian,Greek,Mediterranean,Mexican,Thai,Vietnamese,Others,Review,rating
0,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,This location is out of business. I drove by i...,Low
1,2,2,2,2,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,= = = = = = CLOSED = = = = = =This JB s locati...,Low
2,4,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,This is just a basic (albeit mini) chain greas...,High
3,3,0,1,2,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,Whenever I offer to take my mom out to lunch s...,Low
4,3,7,9,9,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,If I say it wasn t as bad as I was expecting i...,Low


In [3]:
yelp.columns.values

array(['stars', 'votes_cool', 'votes_funny', 'votes_useful', 'Cheap',
       'Moderate', 'Expensive', 'VeryExpensive', 'American', 'Chinese',
       'French', 'Japanese', 'Indian', 'Italian', 'Greek', 'Mediterranean',
       'Mexican', 'Thai', 'Vietnamese', 'Others', 'Review', 'rating'], dtype=object)

#### Task A. Ignore the text (reviews) and run a classification model with the numeric data (you can use standard methods like logistic regression, k-nearest neighbors or anything else). What is the best accuracy of your model with numeric data?

In [4]:
# Take only a random subset of 2000 rows
yelp_subset = yelp.sample(frac=.1,random_state=5)
yelp_subset = yelp_subset.reset_index()
yelp_subset = yelp_subset.drop('index',axis=1)
yelp_subset[:3]

,stars,votes_cool,votes_funny,votes_useful,Cheap,Moderate,Expensive,VeryExpensive,American,Chinese,...,Indian,Italian,Greek,Mediterranean,Mexican,Thai,Vietnamese,Others,Review,rating
0,5,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,I ll be honest...I m not a food connoisseur b...,High
1,2,0,1,3,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,Made reservations for 5:30 PM on a Saturday ni...,Low
2,5,0,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,This place has an awesome Happy Hour. The envi...,High


In [5]:
#split train and test data
from sklearn.cross_validation import StratifiedShuffleSplit 

y = yelp_subset['rating'].map(lambda y: 1 if y == 'High'  else 0)

X = yelp_subset.drop(['stars','Review','rating'], axis=1)

Review = yelp_subset['Review']

sss = StratifiedShuffleSplit(y, test_size=0.3, random_state=1)
for train_index, test_index in sss:
    break

train_x, train_y,train_review = X.values[train_index],y.values[train_index],Review[train_index]
test_x, test_y, test_review = X.values[test_index], y.values[test_index],Review[test_index]

In [6]:
#knn model with numerical inputs only
from sklearn import neighbors
modelknn = neighbors.KNeighborsClassifier(n_neighbors=15, weights='uniform', p=2)
modelknn.fit(train_x,train_y)

from sklearn import metrics
prediction_on_training = modelknn.predict(train_x)
print "Train set accuracy score:", metrics.accuracy_score(train_y, prediction_on_training)

predicted_classes = modelknn.predict(test_x)
print "Test set accuracy score:", metrics.accuracy_score(test_y, predicted_classes)

Train set accuracy score: 0.695714285714
Test set accuracy score: 0.703333333333


In [7]:
#logistic regression with numerical inputs only
from sklearn.linear_model import LogisticRegression
modellog = LogisticRegression()
modellog.fit(train_x, train_y)

from sklearn import metrics
prediction_on_training = modellog.predict(train_x)
print "Train set accuracy score:", metrics.accuracy_score(train_y, prediction_on_training)

predicted_classes = modellog.predict(test_x)
print "Test set accuracy score:", metrics.accuracy_score(test_y, predicted_classes)

Train set accuracy score: 0.692857142857
Test set accuracy score: 0.701666666667


In [8]:
#naive bayes with numerical inputs only
from sklearn import naive_bayes
modelnby = naive_bayes.MultinomialNB()
modelnby.fit(train_x, train_y)

from sklearn import metrics
prediction_on_training = modelnby.predict(train_x)
print "Train set accuracy score:", metrics.accuracy_score(train_y, prediction_on_training)

predicted_classes = modelnby.predict(test_x)
print "Test set accuracy score:", metrics.accuracy_score(test_y, predicted_classes)

Train set accuracy score: 0.687142857143
Test set accuracy score: 0.686666666667


So the best accuracy with using only numeric data for prediction is about 70.3% on test data with k nearest neighbor model.

#### Task B. Perform a supervised classification on a subset of the corpus using the reviews only. You can write your code in Python or R. What accuracy do you get from this text mining exercise?

In [9]:
import operator
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer


import re
import string
import nltk
from nltk import word_tokenize          
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords


stemmer = PorterStemmer()
wnl = WordNetLemmatizer()
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def lemmatize(tokens,wnl):
    lem=[]
    for item in tokens:
        lem.append(wnl.lemmatize(item))
    return lem

def my_tokenizer(text):
    text = "".join([ch for ch in text if ch not in string.punctuation])
    text = re.sub('[0-9]+','',text)
    tokens = nltk.word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    #result = stem_tokens(tokens, stemmer)
    #result = lemmatize(tokens, wnl)
    result = tokens
    return result
    

vectorizer = TfidfVectorizer(min_df=3,max_df=0.9,tokenizer=my_tokenizer,
 ngram_range=(1, 2), 
 stop_words='english',
 strip_accents='unicode',max_features=200)

text_1 = vectorizer.fit_transform(train_review)

In [10]:
from nltk import NaiveBayesClassifier
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB

text_2 = vectorizer.transform(test_review)

nb =  MultinomialNB().fit(text_1, train_y)

y_nb_predicted = nb.predict(text_2)


print "MODEL: Multinomial Naive Bayes\n"

print 'The accuracy for this classifier is ' + str(metrics.accuracy_score(test_y, y_nb_predicted))

MODEL: Multinomial Naive Bayes

The accuracy for this classifier is 0.72


So the accuracy with using only review data for prediction is about 72% on test data with Multinomial Naive Bayes Model.

#### Task C. Combine the numeric data and the text classification model (in task B) to create a “hybrid” model. It is your task to figure out how to do this. Now run this hybrid classification model and compare the results with those in A and B. Does the numeric data add to the predictive power relative to text?

In [11]:
from sklearn.naive_bayes import MultinomialNB
from operator import itemgetter

N = 50
vocabulary = np.array([t for t, i in sorted(vectorizer.vocabulary_.iteritems(), key=itemgetter(1))])

topN = np.argsort(nb.coef_[0])[-N:]

print "\nThe top %d most informative features: \n%s" % (N, ','.join(vocabulary[topN]))


The top 50 most informative features: 
perfect,times,hour,meal,people,rice,right,wait,tacos,mexican,got,sauce,salad,definitely,order,burger,dinner,eat,atmosphere,excellent,cheese,staff,bar,favorite,ordered,phoenix,happy,pizza,awesome,friendly,lunch,try,fresh,little,nice,amazing,restaurant,menu,delicious,chicken,time,really,best,love,like,service,place,good,food,great


In [12]:
most_informative = vocabulary[topN]

for w in most_informative:
    X[w]= 0

def create_features(tokens,i):
    document_words = set(tokens)
    features = {}
    for word in most_informative:
        if word in document_words:
            X.loc[i,word] = 1
        
X.head()

,votes_cool,votes_funny,votes_useful,Cheap,Moderate,Expensive,VeryExpensive,American,Chinese,French,...,time,really,best,love,like,service,place,good,food,great
0,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,3,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
for i in range(1000):
    text = "".join([ch for ch in Review[i] if ch not in string.punctuation])
    text = re.sub('[0-9]+','',text)
    tokens = nltk.word_tokenize(text.lower().decode('utf-8'))
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    feature = create_features(tokens,i)

X.head()

,votes_cool,votes_funny,votes_useful,Cheap,Moderate,Expensive,VeryExpensive,American,Chinese,French,...,time,really,best,love,like,service,place,good,food,great
0,0,0,0,0,1,0,0,1,0,0,...,0,0,0,1,0,0,0,1,1,0
1,0,1,3,0,0,1,0,1,0,0,...,1,0,1,0,1,1,1,1,1,1
2,0,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,1,1,1,0
3,0,0,0,0,0,0,1,1,0,0,...,0,1,0,0,0,1,0,0,0,0
4,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
nb_text =  MultinomialNB().fit(X.values[train_index], train_y)

text_predicted = nb_text.predict(X.values[test_index])


print "Hybrid MODEL: Multinomial Naive Bayes\n"

print 'The accuracy for this classifier is ' + str(metrics.accuracy_score(test_y, text_predicted))

Hybrid MODEL: Multinomial Naive Bayes

The accuracy for this classifier is 0.723333333333


In [15]:
sample_weight = np.random.rand(y.shape[0])

sw_train, sw_test = sample_weight[train_index], sample_weight[test_index]

from sklearn.calibration import CalibratedClassifierCV

from sklearn.metrics import brier_score_loss


clf = MultinomialNB()
model_original = clf.fit(X.values[train_index], train_y)  # GaussianNB itself does not support sample-weights
prob_pos_clf = clf.predict_proba(X.values[test_index])[:, 1]
clf_predicted =model_original.predict(X.values[test_index])
print 'The accuracy for this classifier is ' + str(metrics.accuracy_score(test_y, clf_predicted))

# Naive-Bayes with isotonic calibration
clf_isotonic = CalibratedClassifierCV(clf, cv=10, method='isotonic')
model_isotonic = clf_isotonic.fit(X.values[train_index], train_y, sw_train)
prob_pos_isotonic = clf_isotonic.predict_proba(X.values[test_index])[:, 1]
isotonic_predicted = model_isotonic.predict(X.values[test_index])
print 'The accuracy for this classifier is ' + str(metrics.accuracy_score(test_y, isotonic_predicted))

# Naive-Bayes with sigmoid calibration
clf_sigmoid = CalibratedClassifierCV(clf, cv=10, method='sigmoid')
model_sigmoid  = clf_sigmoid.fit(X.values[train_index], train_y, sw_train)
prob_pos_sigmoid = clf_sigmoid.predict_proba(X.values[test_index])[:, 1]
sigmoid_predicted = model_sigmoid.predict(X.values[test_index])
print 'The accuracy for this classifier is ' + str(metrics.accuracy_score(test_y, sigmoid_predicted))

print("Brier scores: (the smaller the better)")

clf_score = brier_score_loss(test_y, prob_pos_clf, sw_test)
print("No calibration: %1.3f" % clf_score)

clf_isotonic_score = brier_score_loss(test_y, prob_pos_isotonic, sw_test)
print("With isotonic calibration: %1.3f" % clf_isotonic_score)

clf_sigmoid_score = brier_score_loss(test_y, prob_pos_sigmoid, sw_test)
print("With sigmoid calibration: %1.3f" % clf_sigmoid_score)

The accuracy for this classifier is 0.723333333333
The accuracy for this classifier is 0.716666666667
The accuracy for this classifier is 0.706666666667
Brier scores: (the smaller the better)
No calibration: 0.196
With isotonic calibration: 0.196
With sigmoid calibration: 0.200


#### Task D. Use unsupervised sentiment analysis on the reviews (with SentiStrength or any other tool) and use the sentiment scores to predict high/low rating. Compare and contrast the results of tasks B and D. What can you conclude from your analysis?

In [46]:
# Export yelp_subset
yelp_subset.to_csv('yelp_subset.csv',index=False)

# Run sentistrength on column 21 of yelp_subset to get positive and negative sentiments of each review
# Output from sentistrength: yelp_subset_senti+results.txt

# Load back into python
yelp_senti = pd.read_table('yelp_subset_senti+results.txt')
yelp_senti['Overall_senti'] = yelp_senti['Postive_senti'] + yelp_senti['Negative_senti']

# Get predicted ratings
yelp_senti["Pred_rating"] = np.where((yelp_senti['Overall_senti'] > 0),"High","Low")
yelp_senti[:3]

,stars,votes_cool,votes_funny,votes_useful,Cheap,Moderate,Expensive,VeryExpensive,American,Chinese,...,Mexican,Thai,Vietnamese,Others,Review,Rating_final,Postive_senti,Negative_senti,Overall_senti,Pred_rating
0,2,0,1,3,0,0,1,0,1,0,...,0,0,0,0,Made reservations for 5:30 PM on a Saturday ni...,Low,3,-4,-1,Low
1,5,0,1,1,0,1,0,0,1,0,...,0,0,0,0,This place has an awesome Happy Hour. The envi...,High,3,-1,2,High
2,3,0,0,0,0,0,0,1,1,0,...,0,0,0,0,Noted on the OT reservation it was hubby s 50t...,Low,4,-3,1,High


In [48]:
# Checking accuracy
from sklearn.metrics import confusion_matrix
y_true = yelp_senti['Rating_final'].tolist()
y_pred = yelp_senti['Pred_rating'].tolist()
cm_sk = confusion_matrix(y_true, y_pred)

cm = nltk.ConfusionMatrix(y_true, y_pred)
print (cm.pretty_format(sort_by_count=True, show_percents=True))
print 'The accuracy is:',round(float(cm_sk[1][1]+cm_sk[0][0])/float(sum(cm_sk)),3)

     |      H        |
     |      i      L |
     |      g      o |
     |      h      w |
-----+---------------+
High | <54.1%> 14.6% |
 Low |  14.2% <17.2%>|
-----+---------------+
(row = reference; col = test)

The accuracy is: 0.712


#### Task E. Implement the PMI approach to sentiment analysis (in either Python or R), and run the classification model with the sentiment scores. How do your results compare with those in Task D?

In [37]:
import functools
from nltk.corpus import stopwords as stp
from nltk import pos_tag
import requests
from bs4 import BeautifulSoup

In [31]:
def features_extraction(text, rule_1, rule_2, no_match):
    
    wc = []
    append = wc.append
    text = text.lower()
    tokens = re.split("\W+", text)
    stopwords = stp.words("english")
    remove = tokens.remove
    while "" in tokens:
        remove("")
    for token in tokens:
        if token in stopwords:
            remove(token)
    token_pos = pos_tag(tokens)
    
    for i in xrange(len(token_pos)-1):
        if (token_pos[i][1], token_pos[i+1][1]) not in rule_1 and\
            (token_pos[i][1], token_pos[i+1][1]) not in rule_2:
                continue
        elif (token_pos[i][1], token_pos[i+1][1]) in rule_1:
            append((token_pos[i][0], token_pos[i+1][0]))
        elif (token_pos[i][1], token_pos[i+1][1]) in rule_2:
            try:
                if token_pos[i+2] not in no_match:
                    append((token_pos[i][0], token_pos[i+1][0]))
            except IndexError:
                append((token_pos[i][0], token_pos[i+1][0]))
        else:
            continue

    return wc

In [32]:
def Semantic_Orientation(phrases,positive="excellent", negative="poor", prior = 1,engine="google",distance=10,threshold=None):
  
    so_positive = 0.01
    so_negative = 0.01
    so_avg = 0
    get = requests.get

    engine.lower() == "google"
    url = "http://www.google.com/search?q=%s"
    id_pattern = "resultStats"
    
    for phrase in phrases:
        term = "%22{}+{}%22+AROUND({})+%22{}%22".format(phrase[0], distance, phrase[1], positive)
        page = get(url % term)
        soup = BeautifulSoup(page.text, "lxml")
        try:
            rtr_pos = int("".join(re.split("\D+",soup.find("div", id=id_pattern).get_text().encode("utf-８"))))
        except AttributeError:
            rtr_pos = 0
            
        # if rtr_pos == None:
        #     rtr_pos = 0
        
        term = "%22{}+{}%22+AROUND({})+%22{}%22".format(phrase[0], distance, phrase[1], negative)
        page = get(url % term)
        soup = BeautifulSoup(page.text, "lxml")
        try:
            rtr_neg = int("".join(re.split("\D+",soup.find("div", id=id_pattern).get_text().encode("utf-８"))))
        except AttributeError:
            rtr_neg = 0
            
        # if rtr_neg == None:
         #   rtr_neg =0
            
        so_positive += rtr_pos
        so_negative += rtr_neg
        
        so_avg += math.log(so_positive/so_negative/prior)
    
    #if len(phrases)==0:
     #   so_avg=0
    #else:
        so_avg = so_avg*1.0/len(phrases)
    
    if threshold == None:
        return so_avg
    else:
        return int(so_avg>threshold)

In [40]:
# Defining Patters for Feature Extraction
rule_1 = [("JJ", "NN"), ("JJ", "NNS"), ("RB", "VB"), ("RB", "VBD"), ("RB", "VBN"), ("RB", "VBG"),("RBR", "VB"), ("RBR", "VBD"), ("RBR", "VBN"), ("RBR", "VBG"),
          ("RBS", "VB"), ("RBS", "VBD"), ("RBS", "VBN"), ("RBS", "VBG")]
rule_2 = [("RB", "JJ"), ("RBR", "JJ"), ("RBS", "JJ"),("JJ", "JJ"),("NN", "JJ"), ("NNS", "JJ")]
no_match = ["NN", "NNS"]

# Extracting Features

features_extract = functools.partial(features_extraction, rule_1=rule_1, rule_2=rule_2, no_match=no_match)
features=yelp_subset.Review.map(features_extract)

In [41]:
# Calling the function that gives Semantic Orientation for a review.

Opinion_Orientation = functools.partial(Semantic_Orientation,positive="excellent", negative="poor",prior = 1,engine="google",distance=5,threshold=None)

# Mapping the function to our review list(sample :12)

prediction = features[:12].map(Opinion_Orientation)

ValueError: invalid literal for int() with base 10: ''

In [44]:
# Checking accuracy
y_pred = prediction.tolist()
y_true = y[:12].tolist() 


cm_sk = confusion_matrix(y_true, y_pred)
cm = nltk.ConfusionMatrix(y_true, y_pred)


print (cm.pretty_format(sort_by_count=True, show_percents=True))
if cm_sk[1][1] ==None:
    sum_sk[1][1]=0;
print 'The accuracy is:',round(float(cm_sk[1][1]+cm_sk[0][0])/float(len(y_pred)),3)

NameError: name 'prediction' is not defined

#### Task F. What are the top 5 “attributes” of a restaurant that are associated with (i) high and (ii) low ratings? That is, when people rate a restaurant high or low, are they more likely to mention service, ambiance, etc.? 

In [11]:
import nltk
from nltk.corpus import stopwords

#### Performance Tokenization on the positive reviews

In [8]:
pos_reviews  = yelp.ix[yelp.rating == 'High']['Review']
corpus = pos_reviews.str.cat(sep=' ')
corpus = corpus.decode('utf-8')
corpus_words = nltk.word_tokenize(corpus.lower())
corpus_words = [word for word in corpus_words if word.isalpha()==True]

#### Delete stop words and keep only nouns

In [13]:
filtered_words_sw = [word for word in corpus_words if word not in stopwords.words('english')]
filtered_poi = nltk.pos_tag(filtered_words_sw)
noun = [x[0] for x in filtered_poi if x[1] == 'NN']

#### Find the 5 most common nouns in the corpurs, and they are the top attributes

In [28]:
noun = nltk.FreqDist(noun)
noun.most_common(5)

[(u'food', 10462),
 (u'place', 8816),
 (u'service', 4405),
 (u'time', 4148),
 (u'restaurant', 2711)]

#### Perform same analysis on negative reviews

In [30]:
neg_reviews  = yelp.ix[yelp.rating == 'Low']['Review']
neg_corpus = neg_reviews.str.cat(sep=' ')
neg_corpus = neg_corpus.decode('utf-8')
neg_corpus_words = nltk.word_tokenize(neg_corpus.lower())
neg_corpus_words = [word for word in neg_corpus_words if word.isalpha()==True]

In [31]:
neg_words_sw = [word for word in neg_corpus_words if word not in stopwords.words('english')]
neg_poi = nltk.pos_tag(neg_words_sw)
neg_noun = [x[0] for x in neg_poi if x[1] == 'NN']

In [32]:
neg_noun = nltk.FreqDist(neg_noun)
neg_noun.most_common(5)

[(u'food', 6574),
 (u'place', 4370),
 (u'service', 2665),
 (u'time', 2413),
 (u'order', 1788)]